# Problem 2; Reading CSV with good polygons gives bad polygons

Need to find a different way to store our polygons to put into csv...

In [8]:
import pandas as pd
import geopandas as gp
import shapely as sp
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
from itertools import combinations
import intake
from shapely.geometry import MultiPoint

In [9]:
def get_outer_polygon_for_obsid(obsid, return_hull=True):
    tiles = pd.read_csv("tile_coordinates.csv")
    obsid_tiles = tiles.query("obsid == @obsid")
    cols = "BodyFixedCoordinateX BodyFixedCoordinateY".split()
    coords = obsid_tiles[cols]
    coords.columns = ["x", "y"]
    mp = MultiPoint(list(zip(coords.x, coords.y)))
    if return_hull:
        return mp.convex_hull
    else:
        return mp

In [10]:
def interchecker(polygon_array):
    """
    
    """
    bucket2 = []
    combos = []
    iterable1 = range(len(polygon_array)) 
    p = len(polygon_array)
    for n in iterable1:
        if p > 0:
            comb = list(combinations(iterable1,p))
            for item in comb:
                x = item
                combos.append(x)
            p = p-1
        else:
            end

    #the above gives us an array of each possible combination eg
    #combos = [comb[0],comb[1],...]
    #where comb[0] could be (0,1,2,4) or something of the sort
    
    for ele in combos:
        d = {}
        #print(ele) #shows the element used
        #d['region_name'] = region
        #d['index_used'] = ele #for testing
        bucket = []
        for number in ele:
            bucket.append(polygon_array[number])
            #bucket now [poly0, poly1, poly2, poly4] if using comb[0] above
        d['elements_used'] = np.array(bucket)
        if len(bucket)>1:
            y = bucket[0].intersection(bucket[1])
            for i in range(len(bucket)-2):
                y = y.intersection(bucket[i+2])
            #print(y.area)
            d["combo_area"]= y.area
        else:
            #print(bucket[0].area)
            d['combo_area'] = bucket[0].area
        #for number in ele:
            #d['elements_used'] = polygon_array[number]
            #must move elements used to get every element used- where?
        bucket2.append(d)
    return pd.DataFrame(bucket2)

In [11]:
combodf = pd.read_csv('Macclesfield_duplicate_combodf.csv')

In [17]:
combodf.head() #This is actually just our df in specific case, but 
#it has broken polygons! something wrong with reading them in from csv

,combo_area,elements_used,obsids_used,number_hirise,Ls_used,region_name
0,0.008637,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_012079_0945', 'ESP_011407_0945', 'ESP_01...",14.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield
1,0.073851,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_012079_0945', 'ESP_011407_0945', 'ESP_01...",13.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield
2,0.008637,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_012079_0945', 'ESP_011407_0945', 'ESP_01...",13.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield
3,0.008637,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_012079_0945', 'ESP_011407_0945', 'ESP_01...",13.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield
4,0.008637,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_012079_0945', 'ESP_011407_0945', 'ESP_01...",13.0,"(214.785, 183.727, 198.04, 207.892, 235.282, 2...",Macclesfield


In [14]:
combodf.elements_used[0][0]

'['

In [15]:
startest = pd.read_csv("Starburst_duplicate_test_combodf.csv")

In [16]:
startest.head()

,combo_area,elements_used,obsids_used,number_hirise,Ls_used,region_name
0,1.302620,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_011697_0980', 'ESP_011341_0980', 'ESP_01...",10.0,"(196.842, 180.80900000000003, 203.57, 187.2540...",Starburst
1,1.348645,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_011697_0980', 'ESP_011341_0980', 'ESP_01...",9.0,"(196.842, 180.80900000000003, 203.57, 187.2540...",Starburst
2,1.302620,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_011697_0980', 'ESP_011341_0980', 'ESP_01...",9.0,"(196.842, 180.80900000000003, 203.57, 187.2540...",Starburst
3,1.334411,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_011697_0980', 'ESP_011341_0980', 'ESP_01...",9.0,"(196.842, 180.80900000000003, 203.57, 187.2540...",Starburst
4,1.302620,[<shapely.geometry.polygon.Polygon object at 0...,"('ESP_011697_0980', 'ESP_011341_0980', 'ESP_01...",9.0,"(196.842, 180.80900000000003, 203.57, 187.2540...",Starburst
